<svg width="300" height="100" xmlns="http://www.w3.org/2000/svg">
<line x1="10" y1="10" x2="290" y2="10" style="stroke:#000">
</line>
<ellipse cx="150" cy="50" rx="20" ry="20" style="stroke:#000;fill:none;"></ellipse>
</svg>

In [ ]:
from IPython.display import SVG
import svgwrite
import numpy as np
import os
import random
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
import warnings
import cmath
from svgpathtools import *
import math

In [ ]:
from deepsvg.svglib.svg import SVG
from deepsvg.difflib.tensor import SVGTensor

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
paths, attributes, svg_attributes = svg2paths2('Minisample/leaves.svg')

In [ ]:
figures = []
for path in paths:
    wsvg(path,filename='temp_plant.svg')
    plant = SVG.load_svg('temp_plant.svg')
    plant.normalize()
    plant.canonicalize()
    #plant.simplify_heuristic()
    tensor = SVGTensor.from_data(plant.copy().numericalize().to_tensor())
    print(tensor.data)
    tensor.pad(47)
    tensor.add_sos()
    cat_tensor = torch.cat((tensor.control1,tensor.control2,tensor.end_pos),1).unsqueeze(0)
    print(cat_tensor.shape)
    figures.append(cat_tensor)                       

##### Idea: first convert to the line format according to the necessary #of points and then split evenly (smallest element)
##### Hierarchical generation - first generate the structure and then indicidual item
##### conceptual learning

In [ ]:
x = figures[:,:-1,:]
y = figures[:,1:,:]

In [ ]:
x_batches = torch.split(x,50)

In [ ]:
y_batches = torch.split(y,50)

In [ ]:
class LSTMArtist(nn.Module):

    def __init__(self, input_dim, hidden_dim, mix_k, num_layers=2,dropout=0.1): #, dropout=0.1
        super(LSTMArtist, self).__init__()
        
        #self.cov = torch.eye(6)
        #self.cov.requires_grad = True
        
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.linear_mu = nn.Linear(hidden_dim,mix_k*6)
        self.linear_sigmasq = nn.Linear(hidden_dim,1)
        self.linear_mix = nn.Linear(hidden_dim,mix_k)
        
        self.activation = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=2)
        self.relu = nn.ReLU()


    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        coordinates_mu = self.linear_mu(lstm_out) #remove exp in case of delta
        coordinates_sigmasq = torch.exp(self.linear_sigmasq(lstm_out)) #torch.exp self.relu
        coordinates_mix = self.softmax(self.linear_mix(lstm_out))
        return (coordinates_mu,coordinates_sigmasq,coordinates_mix)

In [ ]:
def gaussian_pdf(x, mu, sigmasq):
    # NOTE: we could use the new `torch.distributions` package for this now
    #print((-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2)
    #print(torch.exp((-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2))
    pdf = (1/torch.sqrt(2*np.pi*sigmasq)) * torch.exp(torch.min(torch.tensor(50).to(cuda0),torch.max(torch.tensor(-50).to(cuda0),(-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2)))
    return pdf

In [ ]:
def calculate_loss(coordinates_mu,coordinates_sigmasq,coordinates_mix,k,y):
    losses = torch.zeros(y.shape[0:2],device = cuda0)
    for i in range(k):  # To Do: put k versions for method and absolute, add loss proportionally likelihood
        likelihood_z_x = gaussian_pdf(y, coordinates_mu[:,:,i*6:(i+1)*6], coordinates_sigmasq[:,:, 0])
        #print(likelihood_z_x)
        losses += coordinates_mix[:,:,i] * likelihood_z_x
    return torch.mean(-(torch.log(losses)))

In [ ]:
k = 30
model = LSTMArtist(6,400,k,dropout=0.2,num_layers=2).to(cuda0) #.to(cuda0) ,dropout=0.9
##################
#mse_loss = nn.MSELoss()
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
#################
for epoch in range(20000):
    if epoch % 1000 == 0:
        print("Running epoch",epoch)
    total_loss = 0
    for i,batch in enumerate(x_batches):
        optimizer.zero_grad()
        coordinates_mu,coordinates_sigmasq,coordinates_mix = model(batch)
        #print(coordinates_sigmasq)
        loss = calculate_loss(coordinates_mu,coordinates_sigmasq,coordinates_mix,k,y_batches[i])
        #print(">>>>>>>>>>>>>>>>>>>",losses)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer.step()
        #print(output.size())
        total_loss += loss.item()
    if epoch % 1000 == 0:
        print(total_loss) #with 10 epochs 7720 2.36

In [ ]:
torch.save(model.state_dict(), 'model_MDM_MULti.model')

In [ ]:
def random_sample(coordinates_mu,coordinates_sigmasq,coordinates_mix):
    with torch.no_grad():
        #coordinates_mu = coordinates_mu.cpu()
        #coordinates_sigmasq = coordinates_sigmasq.cpu()
        #coordinates_mix = coordinates_mix.cpu()

        mix_dist = Categorical(coordinates_mix[0,0,:])
        mix_index = mix_dist.sample()
        mu = coordinates_mu[0,0,mix_index*2:(mix_index+1)*2]
        sigma = torch.sqrt(coordinates_sigmasq)
        #sigma = torch.tensor([[[0.00001]]],device=cuda0)
        angle = Normal(mu, sigma).sample()
        #print(str_key,absolute,coordinates)
        return angle

In [ ]:
# Generate the picture from the trained model
def generate_vectors(model,start_vector,iterations):
    vectors = start_vector
    for i in range(iterations):
        coordinates_mu,coordinates_sigmasq,coordinates_mix = model(vectors)
        angle = random_sample(coordinates_mu[:,-1:,:],coordinates_sigmasq[:,-1:,:],coordinates_mix[:,-1:,:])
        vectors = torch.cat((vectors,angle),1)
    return vectors

In [ ]:
start_vector = torch.tensor([[[0.0,0.0]]],device=cuda0)
vectors = generate_vectors(model,start_vector,160)

In [ ]:
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,vectors.shape[1]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+vectors[0,i,:],0)),0)

In [ ]:
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_algo_8.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_algo_8.svg') 